Reading Task Schedule

In [185]:
import pandas as pd

# Read CSV files
tasks_df = pd.read_csv('data/a-tasks.csv')
change_orders_df = pd.read_csv('data/b-change_orders.csv')
risk_events_df = pd.read_csv('data/c-risk_events.csv')
inspections_df = pd.read_csv('data/d-inspection_records.csv')
weather_history_df = pd.read_csv('data/weather_history.csv')
weather_history_2024_df = pd.read_csv('data/weather_history_2024.csv')

# Display the first few rows of each for confirmation
# print("Data:")
# print(tasks_df.head(), "\n")


# Compute priority score
tasks_df['priority_score'] = tasks_df['duration_days'] + tasks_df['type_weight'] + tasks_df['weather_weight']

# type_weight = how important the task is itself.
# weather_weight = how much weather can disrupt it.
# Both come either from engineer judgment or historical project data analysis, and are used in formulas for predicting delays.

# Example scale (1–5):
# 1 → Low impact
# 5 → High impact

# Sort tasks by priority (descending)
# df_sorted = tasks_df.sort_values(by='priority_score', ascending=False)

# Show the priority list
print(tasks_df[['task_id', 'task_name', 'priority_score']])

  task_id               task_name  priority_score
0      T1              Excavation              15
1      T2         Soil Compaction              13
2      T3         Foundation Pour              18
3      T4  Basement Waterproofing              12
4      T5             Backfilling              11
5      T6            Slab Casting              13
6      T7     Ground Floor Column              20
7      T8      Ground Floor Walls              41
8      T9               Roof Slab              49


In [186]:
from datetime import timedelta

# Make a safe copy so original task durations are never lost
tasks_df['base_duration'] = tasks_df['duration_days']
tasks_df['planned_start_date'] = pd.to_datetime(tasks_df['start_date'])

# Apply change orders safely on a separate column
changes_df = change_orders_df

for _, change in changes_df.iterrows():
    task_id = change['task_id']
    added_duration = change['added_duration']

    if task_id in tasks_df['task_id'].values:
        # Only update a separate column (not original duration_days)
        tasks_df.loc[tasks_df['task_id'] == task_id, 'base_duration'] += added_duration
        tasks_df.loc[tasks_df['task_id'] == task_id, 'actual_cost'] += change['cost_impact']
        tasks_df.loc[tasks_df['task_id'] == task_id, 'change_applied'] = change['change_id']
        print(f"✅ Change order {change['change_id']} applied to task {task_id}. Duration increased by {added_duration} days.")
    else:
        print(f"⚠️ Warning: Task {task_id} in change order not found in tasks list.")


# ----------------- Schedule computation -----------------
def compute_schedule(df, change_orders_df, risk_events_df, inspections_df):
    schedule = {}

    for idx, row in df.iterrows():
        dep = row['depends_on']
        base_duration = int(row['duration_days'])  # always from CSV
        duration = base_duration
        print(f"Initial duration for task {row['task_id']} is {duration} days")
        
        # Apply risk score impact
        risk_score = None
        risk = risk_events_df[risk_events_df['task_id'] == row['task_id']]
        if not risk.empty:
            risk_score = float(risk.iloc[0]['risk_score'])
            if risk_score >= 0.7:
                duration += 3
            elif risk_score >= 0.5:
                duration += 2
            elif risk_score >= 0.3:
                duration += 1
        print(f"After risk adjustments: {duration} days (risk_score={risk_score})")

        # Apply inspection failures
        inspection = inspections_df[(inspections_df['task_id'] == row['task_id']) & (inspections_df['passed'] == 0)]
        if not inspection.empty:
            duration += 2  # add 2 days if inspection failed
        print(f"After inspection adjustments: {duration} days")

        # Apply change orders
        changes = change_orders_df[change_orders_df['task_id'] == row['task_id']]
        if not changes.empty:
            for _, change in changes.iterrows():
                duration += int(change['added_duration'])
                row['actual_cost'] += change['cost_impact']  # update cost
                print(f"Change order {change['change_id']} applied: +{change['added_duration']} days, cost impact {change['cost_impact']}")

        # Determine start date
        if pd.isna(dep):
            s_date = pd.to_datetime(row['start_date'])
        else:
            dep_end = schedule[dep]['end_date'] + timedelta(days=1)
            planned_start = pd.to_datetime(row['planned_start_date'])
            print(dep_end, planned_start)
            # Take the later date between dependency end +1 and planned start
            s_date = max(dep_end, planned_start)       
            
        e_date = s_date + timedelta(days=duration - 1)

        schedule[row['task_id']] = {
            'start_date': s_date,
            'end_date': e_date,
            'risk_score': risk_score,
            'adjusted_duration': duration,
            'base_duration': base_duration,
            'actual_cost': row['actual_cost'],
            'planned_start_date' : row['planned_start_date'],
        }

        print(f"Task {row['task_id']} → start: {s_date}, end: {e_date}, duration: {duration}, cost: {row['actual_cost']}")
        print("-------------------------------------")

    return schedule


schedule_map = compute_schedule(tasks_df, change_orders_df, risk_events_df, inspections_df)

# Write results back without touching base_duration
tasks_df['start_date'] = tasks_df['task_id'].apply(lambda x: schedule_map[x]['start_date'])
tasks_df['end_date'] = tasks_df['task_id'].apply(lambda x: schedule_map[x]['end_date'])
tasks_df['risk_score'] = tasks_df['task_id'].apply(lambda x: schedule_map[x]['risk_score'])
tasks_df['adjusted_duration'] = tasks_df['task_id'].apply(lambda x: schedule_map[x]['adjusted_duration'])

tasks_df['change_applied'] = tasks_df['change_applied'].fillna('None')

# Save result
tasks_df.to_csv('result/logically_updated_tasks.csv', index=False)
print("✅ Updated task list with change orders applied.")


✅ Change order C1 applied to task T3. Duration increased by 2 days.
✅ Change order C2 applied to task T6. Duration increased by 1 days.
✅ Change order C3 applied to task T9. Duration increased by 2 days.
Initial duration for task T1 is 7 days
After risk adjustments: 8 days (risk_score=0.4)
After inspection adjustments: 8 days
Task T1 → start: 2024-05-01 00:00:00, end: 2024-05-08 00:00:00, duration: 8, cost: 11500
-------------------------------------
Initial duration for task T2 is 6 days
After risk adjustments: 7 days (risk_score=0.3)
After inspection adjustments: 7 days
2024-05-09 00:00:00 2024-05-08 00:00:00
Task T2 → start: 2024-05-09 00:00:00, end: 2024-05-15 00:00:00, duration: 7, cost: 9500
-------------------------------------
Initial duration for task T3 is 9 days
After risk adjustments: 12 days (risk_score=0.7)
After inspection adjustments: 14 days
Change order C1 applied: +2 days, cost impact 5000
2024-05-16 00:00:00 2024-05-14 00:00:00
Task T3 → start: 2024-05-16 00:00:00, 

In [190]:
# Ensure date columns are datetime
weather_df = weather_history_2024_df
weather_df['date'] = pd.to_datetime(weather_df['time'])
# print(tasks_df['start_date'], weather_df['date'])

# Step 2: Aggregate by date
daily_df = weather_df.groupby(weather_df['date'].dt.date).agg({
    'temperature_2m (°C)': 'mean',
    'weather_code (wmo code)': lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0],
    'rain (mm)': 'sum',
    'snowfall (cm)': 'sum',
    'precipitation_mm': 'mean',
    'visibility (m)': 'min',
    'wind_speed_10m (km/h)': 'mean',
    'wind_speed_80m (km/h)': 'mean',
    'temperature_80m (°C)': 'mean',
    'soil_temperature_0cm (°C)': 'mean',
    'soil_temperature_6cm (°C)': 'mean',
    'soil_temperature_18cm (°C)': 'mean',
    'soil_moisture_0_to_1cm (m³/m³)': 'mean',
    'soil_moisture_1_to_3cm (m³/m³)': 'mean',
    'soil_moisture_3_to_9cm (m³/m³)': 'mean',
    'cloud_cover (%)': 'mean',
    'soil_moisture_9_to_27cm (m³/m³)': 'mean'
}).reset_index().rename(columns={'date': 'date'})


# Rename columns to match the aggregation results
daily_df = daily_df.rename(columns={
    'temperature_2m (°C)': 'temp_mean',
    'weather_code (wmo code)': 'weather_code_mode',
    'rain (mm)': 'rain_sum',
    'snowfall (cm)': 'snowfall_sum',
    'precipitation_mm': 'precipitation_mm',
    'visibility (m)': 'visibility_min',
    'wind_speed_10m (km/h)': 'wind_mean',
    'wind_speed_80m (km/h)': 'wind_80m_mean',
    'temperature_80m (°C)': 'temp_80m_mean',
    'soil_temperature_0cm (°C)': 'soil_temp_0cm_mean',
    'soil_temperature_6cm (°C)': 'soil_temp_6cm_mean',
    'soil_temperature_18cm (°C)': 'soil_temp_18cm_mean',
    'soil_moisture_0_to_1cm (m³/m³)': 'soil_moisture_0_1_mean',
    'soil_moisture_1_to_3cm (m³/m³)': 'soil_moisture_1_3_mean',
    'soil_moisture_3_to_9cm (m³/m³)': 'soil_moisture_3_9_mean',
    'cloud_cover (%)': 'cloud_cover_mean',
    'soil_moisture_9_to_27cm (m³/m³)': 'soil_moisture_9_27_mean'
})

# In real use, replace this with actual delay data
# Let's simulate a delay if rain > 10mm or wind > 15km/h
# Add more weather parameters that might cause construction delays
# Delay if: rain_sum > 10mm, wind_mean > 15km/h, precipitation_mm > 0.5,
# temp_mean < 2°C or temp_mean > 35°C, or weather_code_mode in severe codes (e.g., 80, 95, 99)

# Apart from rain and wind, a few more weather + site-related parameters can influence construction delays:
# Temperature extremes (too hot >35°C, too cold <2°C) → workers’ safety & curing of concrete.
# Snowfall & soil moisture → impacts excavation, foundations, concrete curing.
# Visibility (fog, low light, heavy dust) → impacts crane/lifting & safety.
# Cloud cover → may indicate storms or affect drying.
# Storm/severe weather codes → thunderstorms, heavy rain, hail, extreme winds.
# Humidity/precipitation intensity → affects painting, plaster, roofing.
# Also non-weather factors may need modeling later (e.g., material delivery, labor availability, equipment breakdowns), but for now, your weather aggregation is a good first step.

# Define severe weather codes
severe_weather_codes = [80, 95, 99]  # heavy rain, thunderstorms, hail

# Delay days logic (weighted)
def compute_delay(row):
    delay = 0
    
    # Rain impact
    if row['rain_sum'] > 10:
        delay += 1
    
    # Strong winds
    if row['wind_mean'] > 15:
        delay += 1
    
    # Extreme temperature
    if row['temp_mean'] < 2 or row['temp_mean'] > 35:
        delay += 1
    
    # Heavy snowfall or soil too wet
    if row['snowfall_sum'] > 2 or row['soil_moisture_0_1_mean'] > 0.35:
        delay += 1
    
    # Low visibility
    if row['visibility_min'] < 200:
        delay += 1
    
    # Severe weather code
    if row['weather_code_mode'] in severe_weather_codes:
        delay += 2   # double penalty for storms
    
    return delay

daily_df['delay_days'] = daily_df.apply(compute_delay, axis=1)

print("Daily Weather Data with Delay Days:")
print(daily_df[['date', 'temp_mean', 'rain_sum', 'wind_mean', 'delay_days']].head())

# Save results
daily_df.to_csv('result/daily_weather_aggregated.csv', index=False)

#
#
#
#

Daily Weather Data with Delay Days:
         date  temp_mean  rain_sum  wind_mean  delay_days
0  2024-01-01   6.216667       0.4   9.695833           0
1  2024-01-02   4.804167       7.6   9.716667           0
2  2024-01-03   9.212500       4.1  14.883333           0
3  2024-01-04   4.170833       2.1  14.062500           0
4  2024-01-05   0.541667       1.3  11.166667           2


In [192]:
import pandas as pd
from datetime import timedelta

def apply_weather_delays(tasks_file, weather_file, output_file):
    # Load data
    tasks_df = pd.read_csv(tasks_file, parse_dates=['start_date', 'end_date'])
    weather_df = pd.read_csv(weather_file, parse_dates=['date'])

    severe_weather_codes = [95, 96, 99]  # thunderstorms, hail, etc.

    updated_tasks = []

    for _, row in tasks_df.iterrows():
        task_id = row['task_id']
        s_date = row['start_date']
        e_date = row['end_date']
        duration = int((e_date - s_date).days) + 1  # ensure Python int

        # Get weather conditions during task execution window
        task_weather = weather_df[
            (weather_df['date'] >= s_date) & (weather_df['date'] <= e_date)
        ].copy()

        if not task_weather.empty:
            # Flag delay-causing days
            task_weather['delay'] = (
                (task_weather['rain_sum'] > 10) |
                (task_weather['wind_mean'] > 15) |
                (task_weather['precipitation_mm'] > 0.5) |
                (task_weather['temp_mean'] < 2) |
                (task_weather['temp_mean'] > 35) |
                (task_weather['snowfall_sum'] > 2) |
                (task_weather['visibility_min'] < 500) |
                (task_weather['soil_moisture_0_1_mean'] > 0.45) |
                (task_weather['weather_code_mode'].isin(severe_weather_codes))
            ).astype(int)

            extra_days = int(task_weather['delay'].sum())  # force Python int

            if extra_days > 0:
                print(f"Task {task_id}: +{extra_days} days due to weather")
                duration += extra_days
                e_date = s_date + timedelta(days=int(duration - 1))  # cast to int

        updated_row = row.to_dict()
        updated_row['adjusted_duration'] = duration
        updated_row['adjusted_end_date'] = e_date

        updated_tasks.append(updated_row)

    # Save updated tasks to CSV
    updated_df = pd.DataFrame(updated_tasks)
    updated_df.to_csv(output_file, index=False)
    print(f"\n✅ Weather delays applied. Updated file saved as {output_file}")

if __name__ == "__main__":
    apply_weather_delays(
        tasks_file="result/logically_updated_tasks.csv",
        weather_file="result/daily_weather_aggregated.csv",
        output_file="result/tasks_with_weather_delays.csv"
    )


Task T1: +2 days due to weather
Task T2: +1 days due to weather
Task T3: +2 days due to weather
Task T5: +1 days due to weather
Task T8: +3 days due to weather
Task T9: +11 days due to weather

✅ Weather delays applied. Updated file saved as result/tasks_with_weather_delays.csv
